# Deep Learning

## Assignment 4

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [184]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [185]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:

- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [186]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [187]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.
***
#### Note:
> `tf.nn.conv2d` requires an `(batch_size, height, width, num_channels)` input tensor
> and a `(kernel_height, kernel_width, input_channels, output_channels)` kernel tensor.
> The output is a `(batch_size, out_height, out_width, output_channels)` tensor.
> According to the literature terminology, `channel = kernel = feature map`.
> `out_height` and `out_width` will depend on the stride and padding policy.

In [144]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    #Note: tf.nn.conv2d requires an (batch_size, height, width, num_channels) input tensor
    #and a (kernel_height, kernel_width, input_channels, output_channels) kernel tensor.
    #The output is a (batch_size, out_height, out_width, output_channels) tensor.
    #According to the literature terminology, channel = kernel
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [145]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.978423
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.653244
Minibatch accuracy: 37.5%
Validation accuracy: 48.5%
Minibatch loss at step 100: 0.889566
Minibatch accuracy: 68.8%
Validation accuracy: 68.9%
Minibatch loss at step 150: 0.413528
Minibatch accuracy: 87.5%
Validation accuracy: 75.5%
Minibatch loss at step 200: 0.900337
Minibatch accuracy: 62.5%
Validation accuracy: 78.1%
Minibatch loss at step 250: 1.202480
Minibatch accuracy: 68.8%
Validation accuracy: 76.9%
Minibatch loss at step 300: 0.352090
Minibatch accuracy: 87.5%
Validation accuracy: 78.9%
Minibatch loss at step 350: 0.316963
Minibatch accuracy: 93.8%
Validation accuracy: 76.9%
Minibatch loss at step 400: 0.361247
Minibatch accuracy: 87.5%
Validation accuracy: 80.0%
Minibatch loss at step 450: 0.731405
Minibatch accuracy: 81.2%
Validation accuracy: 79.4%
Minibatch loss at step 500: 0.753689
Minibatch accuracy: 87.5%
Validation accuracy: 79.9%
Mi

***
### Problem 1

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation `(nn.max_pool())` of stride 2 and kernel size 2.

#### Note:

> `tf.nn.max_pool(input, neighborhood_shape, strides, padding)` executes max pooling. `neighbohood_shape` is a list 
> size 4 that specifies the shape of the hyper-rectangular region over which the max is going to be applied. With
> careful design of `neighbohood_shape`, all sorts of pooling behaviors can be accomplished, such as pooling over the 
> same spatial position of several feature maps.
***

In [11]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    #Note: tf.nn.conv2d requires a (batch_size, height, width, num_channels) input tensor
    #and a (kernel_height, kernel_width, input_channels, output_channels) kernel tensor.
    #The output is a (batch_size, out_height, out_width, output_channels) tensor.
    #According to the literature terminology, channel = kernel
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    #max pooling with neighborhood size 5 and stride 2
    pool = tf.nn.max_pool(hidden, [1, 3, 3, 1], [1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, [1, 5, 5, 1], [1, 2, 2, 1], padding='SAME')
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [12]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.405987
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.263523
Minibatch accuracy: 6.2%
Validation accuracy: 19.5%
Minibatch loss at step 100: 2.138855
Minibatch accuracy: 31.2%
Validation accuracy: 34.1%
Minibatch loss at step 150: 1.444383
Minibatch accuracy: 50.0%
Validation accuracy: 48.7%
Minibatch loss at step 200: 1.074754
Minibatch accuracy: 68.8%
Validation accuracy: 73.1%
Minibatch loss at step 250: 1.342936
Minibatch accuracy: 56.2%
Validation accuracy: 73.3%
Minibatch loss at step 300: 0.214822
Minibatch accuracy: 100.0%
Validation accuracy: 81.0%
Minibatch loss at step 350: 0.357493
Minibatch accuracy: 93.8%
Validation accuracy: 80.5%
Minibatch loss at step 400: 0.175677
Minibatch accuracy: 100.0%
Validation accuracy: 82.0%
Minibatch loss at step 450: 0.796877
Minibatch accuracy: 81.2%
Validation accuracy: 79.9%
Minibatch loss at step 500: 0.420923
Minibatch accuracy: 87.5%
Validation accuracy: 82.8%


***
### Problem 2

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.
***

In [188]:
batch_size = 32
patch_size_1 = 3
patch_size_2 = 3
depth_1 = 8
depth_2 = 16
depth_3 = 32
num_hidden_1 = 265
num_hidden_2 = 128

keep_prob = 0.9
l2_penalty = 1e-4
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  #conv layers
  conv1_weights = tf.get_variable(name="w1", shape=[patch_size_1, patch_size_1, num_channels, depth_1],
                              initializer=tf.contrib.layers.xavier_initializer())
  conv1_biases = tf.Variable(tf.zeros([depth_1]))

  conv2_weights = tf.get_variable(name="w2", shape=[patch_size_2, patch_size_2, depth_1, depth_2],
                              initializer=tf.contrib.layers.xavier_initializer())
  conv2_biases = tf.Variable(tf.constant(1.0, shape=[depth_2]))
  
  conv3_weights = tf.get_variable(name="w3", shape=[patch_size_2, patch_size_2, depth_2, depth_3],
                              initializer=tf.contrib.layers.xavier_initializer())
  conv3_biases = tf.Variable(tf.constant(1.0, shape=[depth_3]))
  #fully connected layers
  fc1_weights = tf.get_variable(name="fc1", 
                                shape=[image_size // 4 * image_size // 4 * depth_3, num_hidden_1],
                                initializer=tf.contrib.layers.xavier_initializer())
  fc1_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden_1]))
    
  fc2_weights = tf.get_variable(name="fc2", 
                                shape=[num_hidden_1, num_hidden_2],
                                initializer=tf.contrib.layers.xavier_initializer())
  fc2_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden_2]))
    
  softmax_weights = tf.get_variable(name ="softmax", shape = [num_hidden_2, num_labels],
                                    initializer=tf.contrib.layers.xavier_initializer())

  softmax_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
  
  
  # Model.
  def model(data, use_dropout = False):
    #Note: tf.nn.conv2d requires a (batch_size, height, width, num_channels) input tensor
    #and a (kernel_height, kernel_width, input_channels, output_channels) kernel tensor.
    #The output is a (batch_size, out_height, out_width, output_channels) tensor.
    #According to the literature terminology, channel = feature map
    
    #GabrielLeNet-1
    
    #Conv Layer 1
    conv = tf.nn.conv2d(data, conv1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + conv1_biases)
    #Max pooling with neighborhood size 2 and stride 2
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    if use_dropout:
        pool = tf.nn.dropout(pool, keep_prob)
    
    #Conv Layer 2
    conv = tf.nn.conv2d(pool, conv2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + conv2_biases)
    if use_dropout:
        hidden = tf.nn.dropout(hidden, keep_prob)
    
    #Conv Layer 3
    conv = tf.nn.conv2d(hidden, conv3_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + conv3_biases)
    #Max pooling with neighborhood size 2 and stride 2
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    if use_dropout:
        pool = tf.nn.dropout(pool, keep_prob)
    
    #flatten
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    
    #Fully connected
    hidden = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)
    if use_dropout:
      hidden = tf.nn.dropout(hidden, keep_prob)
    #Fully connected
    hidden = tf.nn.relu(tf.matmul(hidden, fc2_weights) + fc2_biases)
    if use_dropout:
      hidden = tf.nn.dropout(hidden, keep_prob)
    return tf.matmul(hidden, softmax_weights) + softmax_biases
  
  # Training computation.
  logits = model(tf_train_dataset, use_dropout = True)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    #+ l2_penalty*( tf.nn.l2_loss(fc1_weights) + tf.nn.l2_loss(softmax_weights)))
    
  # Optimizer.
  #global_step = tf.Variable(0, trainable=False)
  #starter_learning_rate = 0.1
  #learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 500, 0.96, staircase=True)
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  optimizer = tf.train.AdamOptimizer(1e-3).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [189]:
num_steps = 2001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.174370
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.624551
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 100: 2.059496
Minibatch accuracy: 25.0%
Validation accuracy: 40.0%
Minibatch loss at step 150: 0.698183
Minibatch accuracy: 78.1%
Validation accuracy: 75.2%
Minibatch loss at step 200: 0.547696
Minibatch accuracy: 90.6%
Validation accuracy: 77.7%
Minibatch loss at step 250: 0.912677
Minibatch accuracy: 81.2%
Validation accuracy: 78.4%
Minibatch loss at step 300: 0.467142
Minibatch accuracy: 90.6%
Validation accuracy: 80.8%
Minibatch loss at step 350: 0.770105
Minibatch accuracy: 78.1%
Validation accuracy: 81.8%
Minibatch loss at step 400: 0.775346
Minibatch accuracy: 78.1%
Validation accuracy: 82.2%
Minibatch loss at step 450: 0.703112
Minibatch accuracy: 75.0%
Validation accuracy: 83.0%
Minibatch loss at step 500: 0.615124
Minibatch accuracy: 78.1%
Validation accuracy: 83.0%
Min